In [5]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
from spatialentropy import altieri_entropy, leibovici_entropy
from scipy.stats import entropy
import shapely
import numpy as np
import gc


from tqdm import tqdm
tqdm.pandas()

# select the part to handle
part = 1

In [6]:
gemeenten = gpd.read_parquet(f"data/buurten_parts/buurten_{part}.parquet")


In [7]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

def gm_total_amenities_entropy(gm_name, filter_i, area):
    L0_BLACKLIST, L1_BLACKLIST = ec.getfilter(filter_i)
    amenity_gdf = gpd.read_parquet(f"data/gm_amenities/amenities_{gm_name}.parquet")
    
    amenity_gdf = amenity_gdf[amenity_gdf.within(area)] 
    # apply filters
    amenity_gdf = amenity_gdf[~amenity_gdf.L0_category.isin(L0_BLACKLIST)]
    if L1_BLACKLIST:
        for key, value in L1_BLACKLIST.items():
            amenity_gdf = amenity_gdf[
                ~(
                    (amenity_gdf.L0_category == key)
                    & (amenity_gdf.L1_category.isin(value))
                )
            ]
    
    # total number of amenities
    total_amenities = len(amenity_gdf)
    
    points = [[point.x, point.y] for point in amenity_gdf.geometry]
    
    # calculate entropy
    L0 = amenity_gdf.loc[:, f"L0_category"].values
    L1 = amenity_gdf.loc[:, f"L1_category"].values
    try:
        L0_entropy_lei = leibovici_entropy(points, L0, base=2).entropy
        L0_entropy_alt = altieri_entropy(points, L0, base=2).entropy
    except IndexError:
        L0_entropy_lei = 0
        L0_entropy_alt = 0
    try:
        L1_entropy_lei = leibovici_entropy(points, L1, base=2).entropy
        L1_entropy_alt = altieri_entropy(points, L1, base=2).entropy
    except IndexError:
        L1_entropy_lei = 0
        L1_entropy_alt = 0
    del points
    gc.collect()
    
    return total_amenities, L0_entropy_lei, L1_entropy_lei, L0_entropy_alt, L1_entropy_alt

In [8]:
for filter in [0, 1, 2]:
    print(f"Filter {filter}")
    for i, gm in tqdm(gemeenten.iterrows(), total=len(gemeenten)):
        total_amenities, L0_entropy_lei, L1_entropy_lei, L0_entropy_alt, L1_entropy_alt = gm_total_amenities_entropy(gm["gemeentenaam"], filter, gm["geometry"])
        gemeenten.at[i, f"total_amenities_{filter}"] = total_amenities
        gemeenten.at[i, f"L0_leibovici_{filter}"] = L0_entropy_lei
        gemeenten.at[i, f"L1_leibovici_{filter}"] = L1_entropy_lei
        gemeenten.at[i, f"L0_altieri_{filter}"] = L0_entropy_alt
        gemeenten.at[i, f"L1_altieri_{filter}"] = L1_entropy_alt
        
        del total_amenities, L0_entropy_lei, L1_entropy_lei
        gc.collect()
    
gemeenten.to_parquet(f"data/buurten_parts/calculated/buurten_{part}.parquet")


Filter 0


  2%|▏         | 27/1395 [00:40<11:34,  1.97it/s]/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
  9%|▉         | 124/1395 [01:37<10:57,  1.93it/s]/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w

Filter 1


  2%|▏         | 26/1395 [00:41<14:55,  1.53it/s]/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
  2%|▏         | 27/1395 [00:42<12:31,  1.82it/s]/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w 

Filter 2


  2%|▏         | 26/1395 [00:37<13:19,  1.71it/s]/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
  2%|▏         | 27/1395 [00:37<11:25,  1.99it/s]/home/jleeuwenburgh/miniconda3/envs/scriptie/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w 